In [158]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm

Задание №1. Рассчитайте p-value и коэффициент корреляции на данных Task 1.csv.

Ответ округлите до 4 х знаков после запятой. Какой вывод можно сделать из полученных результатов?

Примечание: данные измерялись в ранговой шкале.

In [65]:
task1 = pd.read_csv('data/Biostatistics_HW3_Task_1.csv')
task1.head(5)

,X,Y
0,94,82
1,84,102
2,122,126
3,56,52
4,34,14


Так как у нас две непрерывные переменные, которые измерялись в ранговой шкале, то рассчитаем коэффициент Спирмена:

In [70]:
result = stats.spearmanr(task1.X, task1.Y)
print('p-value: ', round(result.pvalue, 4))
print('Коэффициент корреляции: ', round(result.statistic, 4))

p-value:  0.0
Коэффициент корреляции:  0.7091


Из полученных результатов можно сделать следующие выводы:
- Значения X и Y имеют высокую корреляцию;
- Так как p-value меньше 0.05, то мы отвергнуть нулевую гипотезу о том, что наблюдаемая связь была получена случайным образом. Таким образом полученная высокая положительная связь между переменными статистически значима.

Задание №2. Постройте модель на основании данных Task 2.csv (доход в зависимости от группы, пола, возраста, образования).

Выведите отчет о построении модели. Чему равен интерсепт полученной модели?

In [3]:
task2 = pd.read_csv('data/Biostatistics_HW3_Task_2.csv')
task2.head(5)

,Субъект №,Группа,Пол,Образование,Возраст,Доход
0,0,Группа 1,Женский,Среднее,51,30000
1,1,Группа 2,Мужской,Учёная степень,40,70000
2,2,Группа 2,Женский,Высшее,40,70000
3,3,Группа 2,Женский,Среднее,31,70000
4,4,Группа 2,Женский,Среднее,42,70000


In [141]:
task2_model = sm.MixedLM.from_formula(
    formula="Доход ~ C(Группа) + C(Пол) + C(Образование) + Возраст", 
    groups="Субъект №", 
    data=task2
).fit()
print('Отчёт о построении модели LMM:')
print(task2_model.summary())

Отчёт о построении модели LMM:
                              Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             Доход         
No. Observations:               120                 Method:                         REML          
No. Groups:                     120                 Scale:                          166183850.6809
Min. group size:                1                   Log-Likelihood:                 -1293.3718    
Max. group size:                1                   Converged:                      Yes           
Mean group size:                1.0                                                               
--------------------------------------------------------------------------------------------------
                                     Coef.          Std.Err.       z    P>|z|   [0.025     0.975] 
--------------------------------------------------------------------------------------------------
Intercept 

In [140]:
print('Значение интерсепта:', task2_model.fe_params.loc['Intercept'])

Значение интерсепта: 45332.26891089592


Задание №3. Протестируйте следующую статистическую гипотезу на основании только что полученной модели.

H0: mu среднее образование = mu учёная степень, 
где mu - среднее значение "Доход" в каждом из уровней образования

Вы принимаете или отклоняете нулевую гипотезу? Почему?

In [181]:
task3_scientist = task2[task2['Образование'] == 'Учёная степень'].copy()
task3_midle = task2[task2['Образование'] == 'Среднее'].copy()

result = stats.ttest_ind(task3_scientist.Доход, task3_midle.Доход)
print('p-value: ', round(result.pvalue, 3))

p-value:  0.933


Так как p-value больше 0.05, то мы можем считать, что две выборки имеют одинаковое распределение. То есть, их средние совпадает.)

Задание №4. Заместите пропущенные данные из файла Task 4.csv методом MMRM.

В качестве ответа выведите № субъекта, временную точку, в которой вы заменили данные и сам результат, округленный до целых.

In [173]:
task4 = pd.read_csv('data/Biostatistics_HW3_Task_4.csv', sep=";")
task4.head(5)

,Субъект №,Пол,Образование,Доход месяц 1,Доход месяц 2,Доход месяц 3,Доход месяц 4,Доход месяц 5,Доход месяц 6
0,1,Мужской,Среднее,40000.0,60000.0,10000.0,40000.0,30000.0,50000.0
1,2,Мужской,Среднее,30000.0,60000.0,10000.0,NaN,40000.0,20000.0
2,3,Мужской,Высшее,40000.0,60000.0,20000.0,40000.0,40000.0,30000.0
3,4,Мужской,Высшее,30000.0,60000.0,10000.0,50000.0,40000.0,40000.0
4,5,Женский,Среднее,40000.0,40000.0,20000.0,70000.0,30000.0,40000.0


Преобразуем дата-сет в другой вид:

In [174]:
task4_transformed = pd.DataFrame(columns=['Субъект №', 'Пол', 'Образование', 'Доход', 'Месяц'])
for row in task4.values:
    for month in range(0, 6):
        task4_transformed.loc[len(task4_transformed)] = [row[0], row[1], row[2], row[3 + month], month + 1]

task4_transformed.head(10)

,Субъект №,Пол,Образование,Доход,Месяц
0,1,Мужской,Среднее,40000.0,1
1,1,Мужской,Среднее,60000.0,2
2,1,Мужской,Среднее,10000.0,3
3,1,Мужской,Среднее,40000.0,4
4,1,Мужской,Среднее,30000.0,5
5,1,Мужской,Среднее,50000.0,6
6,2,Мужской,Среднее,30000.0,1
7,2,Мужской,Среднее,60000.0,2
8,2,Мужской,Среднее,10000.0,3
9,2,Мужской,Среднее,NaN,4


Закодируем категориальные столбцы:

In [175]:
le = LabelEncoder()
le_gender = le.fit_transform(task4_transformed['Пол'])
le_education = le.fit_transform(task4_transformed['Образование'])
task4_transformed[['Пол', 'Образование']] = pd.DataFrame({'Пол': le_gender, 'Образование': le_education})
task4_transformed

,Субъект №,Пол,Образование,Доход,Месяц
0,1,1,1,40000.0,1
1,1,1,1,60000.0,2
2,1,1,1,10000.0,3
3,1,1,1,40000.0,4
4,1,1,1,30000.0,5
...,...,...,...,...,...
955,160,1,0,40000.0,2
956,160,1,0,20000.0,3
957,160,1,0,70000.0,4
958,160,1,0,NaN,5


Удалим значения NaN и опередлим входные параметры для модели:

In [176]:
task_4_y = task4_transformed.dropna()['Доход'].values
task_4_ID = task4_transformed.dropna()['Субъект №'].values
task_4_X = task4_transformed.dropna()[['Пол', 'Образование', 'Месяц']].values

Замещаем пропущенные данные:

In [180]:
task_4_model = sm.MixedLM(task_4_y, task_4_X, groups=task_4_ID).fit()

predicted = pd.DataFrame({'Replaced':
    task_4_model.predict(task4_transformed.loc[task4_transformed['Доход'].isnull(), ['Пол', 'Образование', 'Месяц']]).round()})

df_to_replace = task4_transformed.loc[task4_transformed.index.isin(predicted.index)]
task4_result = df_to_replace.merge(predicted, left_index=True, right_index=True)[['Субъект №', 'Месяц', 'Replaced']]
task4_result.rename(columns={"Replaced": "Доход"})
task4_result

,Субъект №,Месяц,Replaced
9,2,4,43875.0
30,6,1,18512.0
55,10,2,21922.0
58,10,5,32245.0
78,14,1,18512.0
93,16,4,43875.0
162,28,1,33552.0
177,30,4,43905.0
179,30,6,50787.0
234,40,1,18512.0
